In [2]:
!pip install -q -U keras-tuner

In [3]:
import tensorflow as tf
from tensorflow import keras

import keras_tuner as kt

Using TensorFlow backend


In [4]:
(img_train, label_train), (img_test, label_test) = \
    keras.datasets.fashion_mnist.load_data()

# Normalize pixel values
img_train = img_train.astype("float32") / 255.0
img_test = img_test.astype("float32") / 255.0

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int("units", min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation="relu"))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )
    return model

In [7]:
# Use a Hyperband tuner
tuner = kt.Hyperband(model_builder,
                     objective="val_accuracy",
                     max_epochs=10,
                     factor=3,
                     directory="my_dir",
                     project_name="intro_to_kt")

stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

In [8]:
tuner.search(
    img_train, label_train,
    epochs=50, validation_split=0.2,
    callbacks=[stop_early]
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.  The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimzer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 27s]
val_accuracy: 0.8610000014305115

Best val_accuracy So Far: 0.893750011920929
Total elapsed time: 00h 24m 12s

The hyperparameter search is complete.  The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimzer
is 0.001.



In [9]:
# Build the model with the optimal hyperparameters and train it on the data
# for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history["val_accuracy"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print("Best epoch: %d" % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 13s 8ms/step - loss: 0.4960 - accuracy: 0.8237 - val_loss: 0.3850 - val_accuracy: 0.8584
Epoch 2/50
1500/1500 [==============================] - 13s 8ms/step - loss: 0.3690 - accuracy: 0.8655 - val_loss: 0.4030 - val_accuracy: 0.8543
Epoch 3/50
1500/1500 [==============================] - 13s 9ms/step - loss: 0.3322 - accuracy: 0.8776 - val_loss: 0.3432 - val_accuracy: 0.8745
Epoch 4/50
1500/1500 [==============================] - 13s 9ms/step - loss: 0.3050 - accuracy: 0.8864 - val_loss: 0.3308 - val_accuracy: 0.8803
Epoch 5/50
1500/1500 [==============================] - 13s 8ms/step - loss: 0.2869 - accuracy: 0.8942 - val_loss: 0.3436 - val_accuracy: 0.8737
Epoch 6/50
1500/1500 [==============================] - 14s 9ms/step - loss: 0.2683 - accuracy: 0.9001 - val_loss: 0.3214 - val_accuracy: 0.8849
Epoch 7/50
1500/1500 [==============================] - 13s 9ms/step - loss: 0.2577 - accuracy: 0.9048 - val_loss: 0.3198 - val_ac

In [10]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/29
1500/1500 [==============================] - 13s 8ms/step - loss: 0.4926 - accuracy: 0.8249 - val_loss: 0.4042 - val_accuracy: 0.8525
Epoch 2/29
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3696 - accuracy: 0.8644 - val_loss: 0.3873 - val_accuracy: 0.8577
Epoch 3/29
1500/1500 [==============================] - 13s 8ms/step - loss: 0.3313 - accuracy: 0.8781 - val_loss: 0.3481 - val_accuracy: 0.8702
Epoch 4/29
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3033 - accuracy: 0.8871 - val_loss: 0.3296 - val_accuracy: 0.8813
Epoch 5/29
1500/1500 [==============================] - 12s 8ms/step - loss: 0.2868 - accuracy: 0.8934 - val_loss: 0.3446 - val_accuracy: 0.8738
Epoch 6/29
1500/1500 [==============================] - 12s 8ms/step - loss: 0.2713 - accuracy: 0.8983 - val_loss: 0.3382 - val_accuracy: 0.8794
Epoch 7/29
1500/1500 [==============================] - 12s 8ms/step - loss: 0.2553 - accuracy: 0.9055 - val_loss: 0.3298 - val_ac

In [11]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.4440 - accuracy: 0.8868
[test loss, test accuracy]: [0.4439612329006195, 0.8867999911308289]


In [12]:
!ls my_dir/intro_to_kt

!ls my_dir/intro_to_kt/trial_0000

oracle.json  trial_0003  trial_0007  trial_0011  trial_0015  trial_0019  trial_0023  trial_0027
trial_0000   trial_0004  trial_0008  trial_0012  trial_0016  trial_0020  trial_0024  trial_0028
trial_0001   trial_0005  trial_0009  trial_0013  trial_0017  trial_0021  trial_0025  trial_0029
trial_0002   trial_0006  trial_0010  trial_0014  trial_0018  trial_0022  trial_0026  tuner0.json
checkpoint  checkpoint.data-00000-of-00001  checkpoint.index  trial.json
